# Genetic Linear Regression with DuckDB
A solution... via evolution!

(Testing my skillz. Can I do this in SQL?)

In [2]:
import duckdb as db

In [3]:
observations = db.sql('''
WITH RECURSIVE numbers AS (
    -- Start with 0
    SELECT 0 AS num
    UNION ALL
    -- Add 1 to num until we reach 9 (10 rows)
    SELECT num + 1 FROM numbers WHERE num < 9
), 
parameters AS (
    SELECT 
        5 AS mean, 
        2.0 AS stddev  
),
random AS (
    SELECT
    (sqrt(-2 * ln(random())) * cos(2 * pi() * random()) * stddev + mean) AS norm_rand
    FROM numbers, parameters
)

SELECT 
    norm_rand as x,
    3.0 AS b0_true, 
    5.0 AS b1_true,
    b0_true + b1_true*x AS y
FROM random;
''') 

observations

┌────────────────────┬──────────────┬──────────────┬────────────────────┐
│         x          │   b0_true    │   b1_true    │         y          │
│       double       │ decimal(2,1) │ decimal(2,1) │       double       │
├────────────────────┼──────────────┼──────────────┼────────────────────┤
│  4.843973728118579 │          3.0 │          5.0 │ 27.219868640592892 │
│ 5.9930385571355185 │          3.0 │          5.0 │ 32.965192785677594 │
│ 0.6400257904842315 │          3.0 │          5.0 │  6.200128952421157 │
│  6.753611629185032 │          3.0 │          5.0 │  36.76805814592516 │
│  3.720705227405353 │          3.0 │          5.0 │ 21.603526137026765 │
│  4.992406390455309 │          3.0 │          5.0 │ 27.962031952276543 │
│  4.602938364807567 │          3.0 │          5.0 │ 26.014691824037836 │
│  3.283504909399385 │          3.0 │          5.0 │ 19.417524546996923 │
│ 1.2760988288639799 │          3.0 │          5.0 │  9.380494144319899 │
│  2.983003653328475 │          3.0 │ 

In [4]:
betas = db.sql('''
WITH RECURSIVE numbers AS (
    SELECT 0 AS num
    UNION ALL
    SELECT num + 1 FROM numbers WHERE num < 9
), 
init_betas as (
SELECT 
    random()*20 - 10 AS b0,
    random()*20 - 10 AS b1
    FROM numbers
    )
    
SELECT b0, b1, 1 AS generation
FROM init_betas; 
''') 

betas

┌────────────────────┬─────────────────────┬────────────┐
│         b0         │         b1          │ generation │
│       double       │       double        │   int32    │
├────────────────────┼─────────────────────┼────────────┤
│ -7.038059961050749 │  -6.199386976659298 │          1 │
│  -2.43707615416497 │ -3.9147687144577503 │          1 │
│  5.849232585169375 │  1.5177923114970326 │          1 │
│ -2.867250880226493 │ -0.7179676648229361 │          1 │
│ 0.5006530229002237 │  -0.676690530963242 │          1 │
│  8.253279174678028 │    8.32047593779862 │          1 │
│  -5.24256307631731 │  -5.754428654909134 │          1 │
│ 2.0951202791184187 │    3.49586836528033 │          1 │
│  5.921626482158899 │  -4.593592812307179 │          1 │
│    9.4425312243402 │  -9.516047528013587 │          1 │
├────────────────────┴─────────────────────┴────────────┤
│ 10 rows                                     3 columns │
└───────────────────────────────────────────────────────┘

In [5]:
betas

┌────────────────────┬─────────────────────┬────────────┐
│         b0         │         b1          │ generation │
│       double       │       double        │   int32    │
├────────────────────┼─────────────────────┼────────────┤
│ -7.038059961050749 │  -6.199386976659298 │          1 │
│  -2.43707615416497 │ -3.9147687144577503 │          1 │
│  5.849232585169375 │  1.5177923114970326 │          1 │
│ -2.867250880226493 │ -0.7179676648229361 │          1 │
│ 0.5006530229002237 │  -0.676690530963242 │          1 │
│  8.253279174678028 │    8.32047593779862 │          1 │
│  -5.24256307631731 │  -5.754428654909134 │          1 │
│ 2.0951202791184187 │    3.49586836528033 │          1 │
│  5.921626482158899 │  -4.593592812307179 │          1 │
│    9.4425312243402 │  -9.516047528013587 │          1 │
├────────────────────┴─────────────────────┴────────────┤
│ 10 rows                                     3 columns │
└───────────────────────────────────────────────────────┘

In [6]:
last_gen = db.sql('''
SELECT MAX(generation) AS generation FROM betas
''')

last_gen   

┌────────────┐
│ generation │
│   int32    │
├────────────┤
│          1 │
└────────────┘

In [23]:
db.sql('''
SELECT * FROM betas
''')

┌─────────────────────┬─────────────────────┬────────────┐
│         b0          │         b1          │ generation │
│       double        │       double        │   int32    │
├─────────────────────┼─────────────────────┼────────────┤
│  -7.021523867733777 │   5.527854701504111 │          1 │
│   4.014606508426368 │   3.173673152923584 │          1 │
│   4.015446417033672 │ -4.6125719510018826 │          1 │
│  -5.542190824635327 │   7.996660033240914 │          1 │
│  -8.288777275010943 │  -2.249126615934074 │          1 │
│ -7.7329708356410265 │  -7.789027206599712 │          1 │
│   8.133336706086993 │ -1.5540173836052418 │          1 │
│  -5.284641655161977 │  -9.486016831360757 │          1 │
│   5.006641820073128 │  0.6412461493164301 │          1 │
│  -4.642951097339392 │  -6.293677370995283 │          1 │
├─────────────────────┴─────────────────────┴────────────┤
│ 10 rows                                      3 columns │
└───────────────────────────────────────────────────────

In [24]:
last_betas = db.sql('''
select * 
from betas
where generation = (SELECT generation FROM last_gen)
''')

last_betas 

┌─────────────────────┬──────────────────────┬────────────┐
│         b0          │          b1          │ generation │
│       double        │        double        │   int32    │
├─────────────────────┼──────────────────────┼────────────┤
│  -4.703394654206932 │    8.131639817729592 │          1 │
│  -7.374020214192569 │  -3.0283419834449887 │          1 │
│  -2.752887448295951 │   2.4107105704024434 │          1 │
│   4.480965589173138 │   2.0614512637257576 │          1 │
│ -3.3832013700157404 │ -0.17150100320577621 │          1 │
│   8.968714028596878 │   -9.694875725544989 │          1 │
│   9.589133695699275 │  -2.4822022765874863 │          1 │
│  -4.110601749271154 │    9.456729535013437 │          1 │
│   7.740046605467796 │    3.569522202014923 │          1 │
│  3.8345012487843633 │   -2.776635638438165 │          1 │
├─────────────────────┴──────────────────────┴────────────┤
│ 10 rows                                       3 columns │
└───────────────────────────────────────

In [25]:
cart_prod = db.sql('''
SELECT o.*,
    b.b0,
    b.b1,
    b.generation,
    ((b.b0 + b.b1 * o.x) - o.y) ^ 2 AS sq_error
FROM observations as o
CROSS JOIN last_betas as b
''')

cart_prod 

┌────────────────────┬──────────────┬──────────────┬────────────────────┬───────────────────┬────────────────────┬────────────┬────────────────────┐
│         x          │   b0_true    │   b1_true    │         y          │        b0         │         b1         │ generation │      sq_error      │
│       double       │ decimal(2,1) │ decimal(2,1) │       double       │      double       │       double       │   int32    │       double       │
├────────────────────┼──────────────┼──────────────┼────────────────────┼───────────────────┼────────────────────┼────────────┼────────────────────┤
│  5.871597009419422 │          3.0 │          5.0 │  32.35798504709711 │ 7.323397425934672 │  6.807693764567375 │          1 │ 223.12731474928694 │
│  7.514646628772727 │          3.0 │          5.0 │  40.57323314386363 │ 7.323397425934672 │  6.807693764567375 │          1 │ 320.68132402822636 │
│  5.115286195909805 │          3.0 │          5.0 │ 28.576430979549023 │ 7.323397425934672 │  6.807693764

In [26]:
rmse_calc = db.sql('''
SELECT b0, b1, generation, SQRT(AVG(sq_error)) AS rmse
FROM cart_prod
GROUP BY b0, b1, generation
''')

rmse_calc 

┌─────────────────────┬──────────────────────┬────────────┬────────────────────┐
│         b0          │          b1          │ generation │        rmse        │
│       double        │        double        │   int32    │       double       │
├─────────────────────┼──────────────────────┼────────────┼────────────────────┤
│  6.0078212432563305 │   1.2631972273811698 │          1 │  18.20247158418349 │
│ -7.6201093243435025 │    2.032655840739608 │          1 │ 26.191489994745815 │
│   9.670149288140237 │ -0.24914667941629887 │          1 │  23.60961737188277 │
│   8.006762093864381 │   -4.783504814840853 │          1 │  50.03177906518926 │
│   4.278806876391172 │   -2.234309483319521 │          1 │  39.04652957540481 │
│   5.752613008953631 │  -2.6119326427578926 │          1 │  39.88812632076716 │
│   5.957923871465027 │    7.686135713011026 │          1 │  17.52430596137627 │
│ -1.9986214442178607 │  -2.0166311971843243 │          1 │  43.32780675822131 │
│  2.0705318730324507 │   0.

In [45]:
ranked = db.sql('''
SELECT *, 
    rank() over(partition by generation order by rmse) as rank
FROM rmse_calc
ORDER BY rank
''')

ranked 

┌────────────────────┬─────────────────────┬────────────┬────────────────────┬───────┐
│         b0         │         b1          │ generation │        rmse        │ rank  │
│       double       │       double        │   int32    │       double       │ int64 │
├────────────────────┼─────────────────────┼────────────┼────────────────────┼───────┤
│ -6.222380697727203 │   6.746495892293751 │          1 │  3.626395551880176 │     1 │
│ -9.593510329723358 │  6.4530241675674915 │          1 │  5.971943594197365 │     2 │
│ -9.747569593600929 │   8.146180654875934 │          1 │  7.323107065401753 │     3 │
│ -8.500384520739317 │   5.528321359306574 │          1 │  8.865008805692213 │     4 │
│  2.028311872854829 │  2.2664374578744173 │          1 │ 15.992004305794119 │     5 │
│  2.091284766793251 │ 0.08559833746403456 │          1 │  27.96804484438211 │     6 │
│  0.815909062512219 │ 0.12527729384601116 │          1 │ 28.940853696854013 │     7 │
│ -6.768719661049545 │  0.7085528317838907 

In [28]:
db.sql('''
SELECT *, 
    (SELECT generation FROM last_gen) as lastgen
FROM ranked;
''') 

┌─────────────────────┬─────────────────────┬────────────┬────────────────────┬───────┬─────────┐
│         b0          │         b1          │ generation │        rmse        │ rank  │ lastgen │
│       double        │       double        │   int32    │       double       │ int64 │  int32  │
├─────────────────────┼─────────────────────┼────────────┼────────────────────┼───────┼─────────┤
│ -1.1589339328929782 │    6.21058315038681 │          1 │ 2.1490550685389413 │     1 │       1 │
│  2.2462721914052963 │   5.734154041856527 │          1 │  2.601413865656875 │     2 │       1 │
│   6.113454722799361 │   5.215060417540371 │          1 │  4.027994906725334 │     3 │       1 │
│  -9.217716311104596 │   8.221994331106544 │          1 │  5.338630079388203 │     4 │       1 │
│ -2.8464113222435117 │    7.73093291092664 │          1 │  7.102286095465114 │     5 │       1 │
│  -7.146632582880557 │    8.83434598799795 │          1 │   8.53472801173686 │     6 │       1 │
│  -2.39786935970187